In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.linalg import inv, cholesky,svd, null_space 
from helper import *

In [2]:
def get_K_from_w(w):
    '''
    Function to directly obtain the K matrix from w based on equations in the parameters of K.
    Does not perform Cholesky decomposition.
    
    Arguemnts :
    w - IAC
    
    Returns:
    K - camera calibration matrix
    '''
    w_inv = inv(w)
    w_inv = w_inv/w_inv[2,2]
#     print('Inverse of w is :',w_inv)
    ux = w_inv[0,2]
    uy = w_inv[1,2]
    fy = np.sqrt(w_inv[1,1] - uy**2)
    s = (w_inv[0,1] - ux*uy)/fy
    fx = np.sqrt(w_inv[0,0] - s**2 - ux**2)
    K = np.array([[fx,s,ux],[0,fy,uy],[0,0,1]])
    return K

def get_conic_row_constrained(point1,point2):
    '''
    Function to obtain a row of the cnstraint matrix for obtaining w using vanishing points. Assumptions : zero
    skew and square pixels
    '''
    row = np.array([point1[0]*point2[0] +point1[1]*point2[1],(point1[0]*point2[2]+ point1[2]*point2[0]),\
                    (point1[1]*point2[2]+point1[2]*point2[1]), point1[2]*point2[2]])
    return row

def get_conic_constrained(points):
    '''
    Function to obtain the K matrix from vanishing point constraints, under zero skew and square pixels
    assumption
    '''
    # three pairs of vanishing points 
    rows = [get_conic_row_constrained(points[2*i],points[2*i+1]) for i in range(3)]
    rows = np.array(rows)
    c = null_space(rows)
    c = np.squeeze(c).tolist()
    conic = np.array([[c[0],0,c[1]],[0,c[0],c[2]],[c[1],c[2],c[3]]]).reshape((3,3))
    return conic

def get_conic_row(point1,point2):
    '''
    Function to obtain a row of the cnstraint matrix for obtaining w using vanishing points. No assumptions on K.
    '''
    row = np.array([point1[0]*point2[0], (point1[0]*point2[1]+point1[1]*point2[0]), point1[1]*point2[1],\
            (point1[0]*point2[2]+ point1[2]*point2[0]), (point1[1]*point2[2]+point1[2]*point2[1]), point1[2]*point2[2]])
    return row

def get_conic(points):
    '''
    Function to obtain the full K matrix from vanishing point constraints.
    '''
    # five pairs of perpendicular lines 
    rows = np.array([get_conic_row(points[2*i],points[2*i+1]) for i in range(5)])
    c = null_space(rows)
    conic = np.array([[c[0],c[1],c[3]],[c[1],c[2],c[4]],[c[3],c[4],c[5]]]).reshape((3,3))
    return conic

def make_one_row(col1,col2,constrained=False):
    '''
    Function to compute one row of the constraint matrix using homogrpahy relations
    
    Arguments:
    col1 : First column of homography H
    col2 : Second column of homography H
    constrained : if True, zero skew and square pixels assumptions hold
    
    Returns :
    row  - a row of the constraint matrix.
    '''
    row = np.array([col1[0]*col2[0], (col1[0]*col2[1]+col1[1]*col2[0]), col1[1]*col2[1],\
            (col1[0]*col2[2]+ col1[2]*col2[0]), (col1[1]*col2[2]+col1[2]*col2[1]), col1[2]*col2[2]])
    if constrained:
        row  = np.array([col1[0]*col2[0]+col1[1]*col2[1],col1[0]*col2[2]+col1[2]*col2[0],
                         col1[1]*col2[2]+col1[2]*col2[1],col1[2]*col2[2]])
    return row

def get_conic_row_from_H(col1,col2):
    '''
    Computes two rows of the constraint matrix on w from one homography . No assumptions on K
    '''
    row1 = make_one_row(col1,col2)
    row2 = make_one_row(col1,col1) - make_one_row(col2,col2)
    return np.vstack([row1,row2])


def one_conic_row_constrained(col1,col2):
    '''
    Computes two rows of the constraint matrix on w from one homography . Zero skew and square pixels assumption hold
    '''
    row1 = make_one_row(col1,col2,True)
    row2 = make_one_row(col1,col1,True) - make_one_row(col2,col2,True)
    return np.vstack([row1,row2])

def get_conic_from_H(H1,H2,H3):
    '''
    COmputes the IAC given three homographies 
    
    Arguments :
    H1,H2,H3 : homographies defined from the original plane to the given image
    
    returns : conic matrix of shape (3,3)
    '''
    all_rows = []
    for H in [H1,H2,H3]:
        rows = get_conic_row_from_H(H[:,0],H[:,1])
        all_rows.append(rows)
#     print(all_rows[0])
    all_rows = np.vstack(all_rows)
#     print('SHape of all rows : ', all_rows[:-1])
    c = null_space(all_rows[:-1])
#     print('Null space is :\n',c)
    w = np.array([[c[0],c[1],c[3]],[c[1],c[2],c[4]],[c[3],c[4],c[5]]]).reshape((3,3))
    return w


def get_conic_from_H_constrained(H1,H2):
    '''
    COmputes the IAC given two homographies . Zero skew and square pixels assumptions hold.
    
    Arguments :
    H1,H2 : homographies defined from the original plane to the given image
    
    returns : conic matrix of shape (3,3)
    '''
    all_rows = []
    for H in [H1,H2]:
        rows = one_conic_row_constrained(H[:,0],H[:,1])
        all_rows.append(rows)
    all_rows = np.vstack(all_rows)
    c = null_space(all_rows[:-1])
    w = np.array([[c[0],0,c[1]],[0,c[0],c[2]],[c[1],c[2],c[3]]]).reshape((3,3))
    return w
    

## Checkerboard pattern vanishing points

## Image 1
points = np.array([[2392,1730,1],[2810,1850,1],[2292,1900,1], [2722,2025,1]])
lines = [get_line(points[0],points[2]),get_line(points[1],points[3]),get_line(points[0],points[1]),
         get_line(points[2],points[3])]
v_pts = [get_line(lines[0],lines[1]),get_line(lines[2],lines[3])]

points = np.array([[2084,2539,1],[2530,2420,1],[2265,2762,1],[2720,2630,1]])
# points = points//4

## Image 2
lines = [get_line(points[0],points[2]),get_line(points[1],points[3]),get_line(points[0],points[1]),
         get_line(points[2],points[3])]

v_pts.extend([get_line(lines[0],lines[1]),get_line(lines[2],lines[3])])


## Image 3
points = np.array([[2168,2198,1],[2690,2195,1],[2662,2352,1],[3155,2354,1]])
# points = points//4

lines = [get_line(points[0],points[2]),get_line(points[1],points[3]),get_line(points[0],points[1]),
         get_line(points[2],points[3])]

v_pts.extend([get_line(lines[0],lines[1]),get_line(lines[2],lines[3])])

## Image 4
points = np.array([[2742,2209,1],[3236,2216,1],[2779,2509,1],[3290,2515,1]])
# points = points//4
lines = [get_line(points[0],points[2]),get_line(points[1],points[3]),get_line(points[0],points[1]),
         get_line(points[2],points[3])]

v_pts.extend([get_line(lines[0],lines[1]),get_line(lines[2],lines[3])])

## Image 5
points = np.array([[3491,1855,1],[3905,1942,1],[3217,2356,1],[3621,2455,1]])
# points = points
lines = [get_line(points[0],points[2]),get_line(points[1],points[3]),get_line(points[0],points[1]),
         get_line(points[2],points[3])]

v_pts.extend([get_line(lines[0],lines[1]),get_line(lines[2],lines[3])])


np.set_printoptions(precision=2,suppress=True)
conic = get_conic(v_pts)
K = inv(cholesky(conic)) # Note that inverting the order of inverse and cholesky operations can lead to incorrect results
print('================')
print('Answer for question 1a:\n',K)


points11 = np.array([[2496,1560,0,1],[3304,1792,1,1],[2304,1904,0,0],[3160,2152,1,0]])
points12 = np.array([[2742,2209,0,1],[3236,2216,1,1], [2779,2509,0,0],[3290,2515,1,0]])
points13 = np.array([[3491,1855,0,1],[3905,1942,1,1],[3621,2455,1,0],[3217,2356,0,0]])

H1 = get_H(points11)
H2 = get_H(points12)
H3 = get_H(points13)


w = get_conic_from_H(inv(H2),inv(H3),inv(H1))
K_inv = cholesky(w)
K = inv(K_inv)
print('================')
print('Answer for question 1b :\n',K/K[2,2])
### Uncomment to obtain K directly from w without cholesky 
# print('Answer for question 1b:\n',get_K_from_w(w))


w = get_conic_constrained(v_pts[4:])
K_inv = cholesky(w)
K = inv(K_inv)
K = K/K[2,2]
# print('Answer for question 2a:\n',K)
print('================')
print('Answer for question 2a:\n',get_K_from_w(w))

w = get_conic_from_H_constrained(inv(H1),inv(H2)).astype(float)
# print(type(w[0,0]))
K_inv = cholesky(w)
w_inv = inv(w)
w_inv = w_inv/w_inv[2,2]
K = inv(K_inv)
K = K/K[2,2]
print('================')
print('Answer for question 2b:\n ',K)
# print('Answer for question 2b:\n',get_K_from_w(w))




Answer for question 1a:
 [[6791.65 -392.12  185.02]
 [   0.   2978.96 -572.98]
 [   0.      0.      1.02]]
[[0 1]
 [1 1]
 [0 0]
 [1 0]]
[[0 1]
 [1 1]
 [0 0]
 [1 0]]
[[0 1]
 [1 1]
 [1 0]
 [0 0]]
Answer for question 1b :
 [[ 7876.5  -1035.55   858.89]
 [    0.    8809.45  7755.87]
 [    0.       0.       1.  ]]
Answer for question 2a:
 [[7687.76    0.   2723.01]
 [   0.   7687.76 8592.86]
 [   0.      0.      1.  ]]
Answer for question 2b:
  [[13531.35    -0.    2995.79]
 [    0.   13531.35  1141.62]
 [    0.       0.       1.  ]]


In [3]:
# get_conic_row_constrained([1,1,1],[2,2,2])

In [4]:
# list1 = [[2.58391714e+05, 7.57538571e+04, 1.00000000e+00],[5.18928859e+03, -3.07345638e+03,  1.00000000e+00],
#          [ 1.39116332e+04, -6.16803469e+02,  1.00000000e+00],[-2.74280505e+03, -3.40783716e+03,  1.00000000e+00],
#         [-1.33355862e+04,  2.28710107e+03,  1.00000000e+00], [8.17868945e+03, 4.07177768e+03, 1.00000000e+00],
#          [1.26103390e+05, 3.95703589e+03, 1.00000000e+00], [1.68084836e+03, -6.39493226e+03,  1.00000000e+00],
#         [-1.27860504e+04, -1.56553957e+03,  1.00000000e+00],[-3.42341220e+04,  7.08341463e+04,  1.00000000e+00]]

# points  = np.array([])
# # points1 = np.array([[2811,1851,2800,1800],[2396,1731,2400,1800],[2296,1903,2400,], [2726,2027,1]])
# # points2  = np.array([[2530,2420,2500,2400],[2720,2630,1], [2265,2762,1], [2084,2539,1]])
# points3  = np.array([[2690,2195,1],[2168,2198,1],[2662,2352,1],[3155,2354,1]])
# points4 = np.array([[3236,2216,1],[2742,2209,1],[2779,2509,1],[3290,2515,1]])


In [5]:



# points = np.array([[2396,1731,1],[2811,1851,1],[2296,1903,1], [2726,2027,1]])
# points = np.array([[2168,2198,1],[2690,2195,1],[2662,2352,1],[3155,2354,1]])

# points11 = np.array([[2168,2198,0,1],[2690,2195,1,1], [2662,2352,0,0],[3155,2354,1,0]])
# points12 = np.array([[2742,2209,0,1],[3236,2216,1,1], [2779,2509,0,0],[3290,2515,1,0]])
# # points13 = np.array([[2168,2198,2000,2000],[2690,2195,2400,2000],[2662,2352,2000,2400],[3155,2354,2400,2400]])
# # points13 = np.array([[2742,2209,2500,2000],[3236,2216,3000,2000],[2779,2509,2500,2500],[3290,2515,3000,2500]])
# points13 = np.array([[3491,1855,0,1],[3905,1942,1,1],[3621,2455,1,0],[3217,2356,0,0]])

# H1 = get_H(points11)
# H2 = get_H(points12)
# H3 = get_H(points13)
# # print('H1',H1/H1[2,2])
# # print('H2',H2/H2[2,2])
# # print('H3',H3/H3[2,2])


# w = get_conic_from_H(inv(H2),inv(H3),inv(H1))
# print(w)
# K = cholesky(w)
# # u,d,_ = svd(w)
# print(inv(w))
# # K_inv = u@np.sqrt(np.diag(d))
# K = get_K_from_w(w)
# # K = K/K[2,2]
# print(K)

# print('original \n',w,'from K ',inv(K@K.T))

In [6]:
img2_2 = cv2.imread('./Images/image2_2.png')
points21= np.array([[55.5,106.5,50,100],[396.0,31.5,300,100],[124.5,379.5,50,350],[403.5,294,300,350]])
points22= np.array([[517.5,39,50,100],[829.5,156,300,100],[516,297,50,350],[766.5,427.5,300,350]])
points23 = np.array([[406.5,357,50,100],[705.0,436.5,300,100],[255.0,577.5,50,350],[613.5,700.5,300,350]])

# pointso = np.array([[50,100,1],[300,100,1],[50,350,1],[300,350,1]]) #original 
H1 = get_H(points21)
H2 = get_H(points22)
H3 = get_H(points23)
# print(H1,'\n',H2,'\n',H3)
  
w = get_conic_from_H(inv(H1),inv(H2),inv(H3))
# print(inv(w))
K = inv(cholesky(w))
K = K/K[2,2]
print('Answer for question 1b\n:' ,K)


lines = [get_line(points21[0,:2],points21[1,:2]),get_line(points21[2,:2],points21[3,:2]),\
         get_line(points21[0,:2],points21[2,:2]),get_line(points21[1,:2],points21[3,:2])]
v_pts = [get_line(lines[0],lines[1]),get_line(lines[2],lines[3])]


lines = [get_line(points22[0,:2],points22[1,:2]),get_line(points22[2,:2],points22[3,:2]),\
         get_line(points22[0,:2],points22[2,:2]),get_line(points22[1,:2],points22[3,:2])]
v_pts.extend([get_line(lines[0],lines[1]),get_line(lines[2],lines[3])])

lines = [get_line(points23[0,:2],points23[1,:2]),get_line(points23[2,:2],points23[3,:2]),\
         get_line(points23[0,:2],points23[2,:2]),get_line(points23[1,:2],points23[3,:2])]
v_pts.extend([get_line(lines[0],lines[1]),get_line(lines[2],lines[3])])

conic = get_conic_constrained(v_pts)
K = inv(cholesky(conic))
K = K/K[2,2]
print('Answer to question 2a is :\n',K)

w = get_conic_from_H_constrained(inv(H1),inv(H2))
K = inv(cholesky(w))
K = K/K[2,2]
print('Answer to question 2b is :\n',K)



[[ 50. 100.]
 [300. 100.]
 [ 50. 350.]
 [300. 350.]]
[[ 50. 100.]
 [300. 100.]
 [ 50. 350.]
 [300. 350.]]
[[ 50. 100.]
 [300. 100.]
 [ 50. 350.]
 [300. 350.]]
Answer for question 1b
: [[1126.78  -13.95  456.61]
 [   0.   1128.99  347.13]
 [   0.      0.      1.  ]]
Answer to question 2a is :
 [[1132.22   -0.    440.04]
 [   0.   1132.22  382.98]
 [   0.      0.      1.  ]]
Answer to question 2b is :
 [[1127.54   -0.    440.84]
 [   0.   1127.54  335.16]
 [   0.      0.      1.  ]]


In [7]:
# point_ = np.array([points22[1,0],points22[1,1],1]).reshape(-1)
# print(point_)
# a = H2@point_ 
# print(a/a[-1])


# conic_cons =  get_conic_constrained(v_pts[:6])
# # print(inv(conic_cons))
# K_inv = cholesky(conic_cons)
# K = inv(K_inv)

# # def get_conic_constrained_from_H(H1,H2):
    
# print(K)

In [8]:
### Img2.jpg 

points2 = np.array([[3136,2112],[5392,968],[3128,3160],[5232,1960]])
points2_2 = np.array([[3132,2112],[1600,880],[3128,3160],[1488,1728]])
lines = [get_line(points2[0],points2[1]),get_line(points2[2],points2[3]),get_line(points2[0],points2[2]),
         get_line(points2[1],points2[3]),get_line(points2_2[0],points2_2[1]),
        get_line(points2_2[2],points2_2[3])]

v_pts = [get_line(lines[2*i],lines[2*i+1]) for i in range(3)]
print(v_pts)
v_pts.extend(v_pts)
conic_cons = get_conic_constrained(v_pts)
K_inv = cholesky(conic_cons)
K2 = inv(K_inv)
K2 = K2/K2[2,2]
print('================')
np.set_printoptions(precision=10,suppress=True)
print('Camera calibration matrix using 2a : \n{}'.format(K2))
# print()




[array([19633.03, -6253.52,     1.  ]), array([ 3033.09, 15593.24,     1.  ]), array([-12108.53, -10144.09,      1.  ])]
Camera calibration matrix using 2a : 
[[11784.3045767713    -0.            4628.9118959994]
 [    0.           11784.3045767713  2573.6085492046]
 [    0.               0.               1.          ]]


In [9]:
### img1.jpg

h_points = np.array([[11,1265],[2024,2101],[11,4147],[2024,4532]])
h_points2 = np.array([[2024,2079],[3949,616],[2024,4510],[3949,3773]])
v_points = np.array([[2024,2079],[2024,4510],[2321,4598],[2332,5918]])

lines = [get_line(h_points[0],h_points[1]),get_line(h_points[2],h_points[3]),get_line(h_points2[0],h_points2[1]),
        get_line(h_points2[2],h_points2[3]),get_line(v_points[0],v_points[1]),get_line(v_points[2],v_points[3])]
v_pts = [get_line(lines[2*i],lines[2*i+1]) for i in range(3)]
v_pts.extend(v_pts)

conic_cons = get_conic_constrained(v_pts)
K_inv = cholesky(conic_cons)
K = inv(K_inv)
K = K/K[2,2]
print('Camera calibration matrix using 2a : \n',K)

Camera calibration matrix using 2a : 
 [[8317.789841138    -0.           2794.0484022391]
 [   0.           8317.789841138  4898.2068482037]
 [   0.              0.              1.          ]]
